In [46]:
# 네이버 검색 API 예제 - 블로그 검색
# get_naver_shopping_api_data.ipynb에서 추출

import os
import sys
import urllib.parse
import urllib.request
import json
import re

def set_naver_api_client_address():
    client_id = "m4qSqBkoO9DyCqqPDtoe"
    client_secret = "BP2d8q1NLY"
    return client_id, client_secret

def set_naver_api_search_role(search_word, start=1 ,type="json"):
    encText = urllib.parse.quote(search_word)
    if type in ["json", "xml"]:
        url = f"https://openapi.naver.com/v1/search/shop.{type}?query=" + encText + f"&display=100&start={start}" # JSON 결과
        return url
    else:
        raise print("지원하지 않는 타입의 확장자입니다. 지원타입: [json, xml]")

def set_request_options(url, client_id, client_secret):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    return rescode, response

def save_search_result(rescode, response, search_word, start=1, type="json"):
    if rescode == 200:
        response_body = response.read()
        
        # JSON 데이터 파싱
        data = json.loads(response_body.decode('utf-8'))
        
        # 중고 및 공기계 필터링 후 <b> 태그 제거
        filtered_items = []
        for item in data['items']:
            # 중고 및 공기계 필터링
            if "공기계" not in item.get('category3', "") and "중고" not in item.get('category3', ""):
                # <b> 태그 제거
                item['title'] = re.sub(r"<.*?>", "", item['title'])
                filtered_items.append(item)

        items_title_name = change_search_word_ko_to_en(search_word)
        
        # 필터링된 데이터 JSON 파일로 저장
        with open(f'naver_data_products/naver_api_response_{items_title_name}_{start}.{type}', 'w', encoding='utf-8') as f:
            json.dump(filtered_items, f, ensure_ascii=False, indent=2)
        
        print(f"필터링 후 {type.upper()} 파일이 저장되었습니다.")
    else:
        print("Error Code:", rescode)

from enum import Enum

class ProductMainName(Enum):
    SMARTPHONE = "smartphone"
    TABLETPC = "tabletpc"
    LAPTOP = "laptop"

def change_search_word_ko_to_en(search_word):
    if search_word in ["노트북", "랩탑", "렙탑"]:
        return ProductMainName.LAPTOP.value
    elif search_word in ["스마트폰", "휴대폰"]:
        return ProductMainName.SMARTPHONE.value
    elif search_word in ["테블릿PC", "태블릿PC", "테블릿pc", "태블릿pc", "갤탭", "태블릿", "테블릿"]:
        return ProductMainName.TABLETPC.value
    
def get_naver_shopping_api_data_pipeline(search_word, start):
    client_id, client_secret = set_naver_api_client_address()
    url = set_naver_api_search_role(search_word, start)
    rescode, response = set_request_options(url, client_id, client_secret)
    save_search_result(rescode, response, search_word, start)

def run_naver_api(search_word = "스마트폰", start=1):
    if __name__=="__main__":
        get_naver_shopping_api_data_pipeline(search_word, start)

In [47]:
# python to mysql products에서 추출

import mysql.connector

def connet_db():
    # MySQL 데이터베이스 연결
    db_connection = mysql.connector.connect(
        host="influencerdb.c18uiyu26mws.ap-northeast-2.rds.amazonaws.com",  # MySQL 서버 주소
        user="admin",       # MySQL 사용자 이름
        password="root12#$",  # MySQL 사용자 비밀번호
        database="finaldatabase"  # 사용할 데이터베이스 이름
    )
    return db_connection

def get_cursor(db_connection):
    # 커서 객체 생성
    cursor = db_connection.cursor()
    return cursor

# 데이터를 삽입할 함수 정의
def insert_products_data(db_connection, cursor, category_id, product_name, brand, model):
    # SQL 쿼리 준비
    # product_name : title, brand : maker, model : brand
    insert_query = "INSERT INTO products (category_id, product_name, brand, model) VALUES (%s, %s, %s, %s)"
    select_query = "SELECT COUNT(*) FROM products WHERE category_id = %s AND product_name = %s AND brand = %s AND model = %s"
    values = (category_id, product_name, brand, model)

    
    # 중복 데이터 확인
    cursor.execute(select_query, values)
    count = cursor.fetchone()[0]
    if count == 0:  # 중복되지 않은 경우에만 삽입
        # 쿼리 실행
        cursor.execute(insert_query, values)

        # 변경 사항 저장
        db_connection.commit()
        print(f"Inserted: {category_id} - {product_name} - {brand} - {model}")
    else:
        print(f"Skipped (Duplicate): {category_id} - {product_name} - {brand} - {model}")


def set_category_id(category):
    if category == '노트북':
        return 1
    elif category == '태블릿PC':
        return 3
    elif category == '휴대폰':
        return 2
    else:
        raise print("카테고리에 없는 제품입니다.")
    

def set_json_file(product_main_name, start=1, type="json"):
    import json
    import xml
    # 파일 경로를 지정하여 JSON 파일을 엽니다.
    file_path = f'naver_data_products/naver_api_response_{product_main_name}_{start}.{type}'
    # JSON 파일을 로드하는 방법
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_category(data, count):
    return data[count]["category2"]

def get_title(data, count):
    title = data[count]["title"].split()[:6]
    return " ".join(title)

def get_brand(data, count):
    return data[count]["maker"]

def get_model(data, count):
    return data[count]["brand"]

from enum import Enum

class ProductMainName(Enum):
    SMARTPHONE = "smartphone"
    TABLETPC = "tabletpc"
    LAPTOP = "laptop"

# # Enum 값 출력
# print(f"Smartphone Key: {ProductMainName.SMARTPHONE.name}, Value: {ProductMainName.SMARTPHONE.value}")
# print(f"TabletPC Key: {ProductMainName.TABLETPC.name}, Value: {ProductMainName.TABLETPC.value}")
# print(f"Laptop Key: {ProductMainName.LAPTOP.name}, Value: {ProductMainName.LAPTOP.value}")

def change_search_word_ko_to_en(search_word):
    if search_word in ["노트북", "랩탑", "렙탑"]:
        return ProductMainName.LAPTOP.value
    elif search_word in ["스마트폰", "휴대폰"]:
        return ProductMainName.SMARTPHONE.value
    elif search_word in ["테블릿PC", "태블릿PC", "테블릿pc", "태블릿pc", "갤탭", "태블릿", "테블릿"]:
        return ProductMainName.TABLETPC.value

def connect_python_to_mysql(search_word, start):
# 예시 데이터 입력
    product_main_name = change_search_word_ko_to_en(search_word)
    db_connection = connet_db()
    cursor = get_cursor(db_connection)
    data = set_json_file(product_main_name, start)
    return db_connection, cursor, data

def put_data_into_mysql(db_connection, cursor, data):
    for i in range(len(data)): # count = product_id
        category_id = set_category_id(get_category(data, count=i))
        product_name = get_title(data, count=i)
        brand = get_brand(data, count=i)
        model = get_model(data, count=i)
        insert_products_data(db_connection, cursor, category_id, product_name, brand, model)

def close_mysql_connect(db_connection, cursor):
    # 커서와 연결 종료
    cursor.close()
    db_connection.close()

def python_to_mysql_products_pipeline(search_word, start):
    db_connection, cursor, data = connect_python_to_mysql(search_word, start)
    put_data_into_mysql(db_connection, cursor, data)
    close_mysql_connect(db_connection, cursor)

def run_connect_python_to_mysql_for_products_data(search_word, start):
    if __name__=="__main__":
        python_to_mysql_products_pipeline(search_word, start)

In [48]:
# python_to_mysql_specification

import mysql.connector

def connet_db():
    # MySQL 데이터베이스 연결
    db_connection = mysql.connector.connect(
        host="localhost",  # MySQL 서버 주소
        user="root",       # MySQL 사용자 이름
        password="root1234",  # MySQL 사용자 비밀번호
        database="Final_project_team4"  # 사용할 데이터베이스 이름
    )
    return db_connection

def get_cursor(db_connection):
    # 커서 객체 생성
    cursor = db_connection.cursor()
    return cursor

# 데이터를 삽입할 함수 정의
def insert_specification_data(db_connection, cursor, product_main_name, product_id, spec_name, spac_value):
    # SQL 쿼리 준비
    query = f"INSERT INTO Specifications_{product_main_name} (product_id, spec_name, spac_value) VALUES (%s, %s, %s)"
    values = (product_id, spec_name, spac_value)

    # 쿼리 실행
    cursor.execute(query, values)

    # 변경 사항 저장
    db_connection.commit()
    print(f"Data inserted: {product_id}, {spec_name}, {spac_value}")


def find_product_id(cursor, product_name):
    # 특정 제품 이름으로 product_id 조회
    query = "SELECT product_id FROM products WHERE product_name = %s"
    cursor.execute(query, (product_name,))

    # 결과 가져오기
    result = cursor.fetchone()

    if result:
        product_id = result[0]
        print(f"The product_id for {product_name} is: {product_id}")
        return product_id
    else:
        raise print(f"No product found with the name '{product_name}'.")
        

##################################################################################################################################

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

def get_json_data(product_main_name, start, type="json"):
    # 파일 경로를 지정하여 JSON 파일을 엽니다.
    file_path = f'naver_data_products/naver_api_response_{product_main_name}_{start}.{type}'

    # JSON 파일을 로드하는 방법
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_link(product_main_name:str, count:int=0, start=1):
    data = get_json_data(product_main_name, start)
    # 리턴할 링크 넘기기
    link = data[count]["link"]
    print(link)
    return link

def load_page(link):
    # 페이지 로드에 약 7~8초 걸림
    driver = webdriver.Chrome()
    url = link
    driver.get(url)
    driver.implicitly_wait(8)
    driver.refresh()
    driver.implicitly_wait(8)
    return driver

# 페이지 로드 대기
# time.sleep(5)

def click_butten(driver, xpath='//*[@id="container"]/div[2]/div[1]/div[3]/a'):
    # 버튼을 XPath로 찾아 클릭하기
    try:
        button = driver.find_element(By.XPATH, xpath)  # XPath로 버튼 요소 선택
        button.click()  # 버튼 클릭
        print("버튼을 클릭했습니다.")
    except Exception as e:
        raise print("버튼을 찾을 수 없거나 클릭할 수 없습니다.")

def get_data(driver):
    description_content=[]
    for j in range(1, 4):
        # 반복문 내에서 요소를 찾는 코드에 응답 대기 시간 적용
        try:
            for i in range(1, 12):
                # 최대 10초까지 대기
                row = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="section_spec"]/div[{j}]/table[{i}]/tbody/tr'))
                )
                # description_content.append(meta_description.text.split())
                # tr 안의 th 요소 가져오기
                th_element = row.find_element(By.TAG_NAME, 'th')
                th_text = th_element.text

                # tr 안의 td 요소 가져오기
                td_element = row.find_element(By.TAG_NAME, 'td')
                td_text = td_element.text

                # th, td 각각 저장
                description_content.append({'header': th_text, 'value': td_text})
        except:
            pass
        if description_content:
            return description_content

def quit_driver(driver):
    # 드라이버 종료
    driver.quit()

def scroll_page(driver):
    # 현재 스크롤 위치 가져오기
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    # 현재 위치에서 y 좌표 200px 만큼 아래로 스크롤하기
    scroll_distance = 7000  # 이동할 거리
    new_scroll_position = current_scroll_position + scroll_distance
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")

def click_butten_smartstore(driver, xpath='//*[@id="_productFloatingTab"]/div/div[3]/ul/li[1]/a'):
    # 버튼을 XPath로 찾아 클릭하기
    try:
        button = driver.find_element(By.XPATH, xpath)  # XPath로 버튼 요소 선택
        button.click()  # 버튼 클릭
        print("버튼을 클릭했습니다.")
    except Exception as e:
        print("버튼을 찾을 수 없거나 클릭할 수 없습니다.")
    time.sleep(1)

def get_data_smartstore(driver):
    for j in range(3, 6):
        description_content=[]
        # 반복문 내에서 요소를 찾는 코드에 응답 대기 시간 적용
        try:
            # 최대 1초까지 대기
            for i in range(1, 12):
                row = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="INTRODUCE"]/div/div[{j}]/div/div[2]/div/table/tbody/tr[{i}]'))
                )
                # description_content.append(meta_description.text)
                # tr 안의 th 요소 가져오기
                th_element = row.find_element(By.TAG_NAME, 'th')
                th_text = th_element.text

                # tr 안의 td 요소 가져오기
                td_element = row.find_element(By.TAG_NAME, 'td')
                td_text = td_element.text

                # th, td 각각 저장
                description_content.append({'header': th_text, 'value': td_text})
        except:
            pass
        if description_content:
            return description_content
        

def get_title(data, count):
    title = data[count]["title"].split()[:6]
    return " ".join(title)

def get_detail_spec_from_selenium(search_word, count, start):
    product_main_name = change_search_word_ko_to_en(search_word)
    link = get_link(product_main_name ,count, start)
    driver = load_page(link)
    data = get_json_data(product_main_name, start)
    product_name = get_title(data, count)
    if not "smartstore" in link:
        click_butten(driver=driver)
        description_content = get_data(driver)
        # description_content.insert(0, {'title' : product_name})
    elif "smartstore" in link:
        scroll_page(driver)
        click_butten_smartstore(driver)
        description_content = get_data_smartstore(driver)
        # description_content.insert(0, {'title' : product_name})
    else:
        pass
    quit_driver(driver)
    return product_name, description_content

def insert_selenium_data_to_mysql(search_word, product_name, description_content):
    product_main_name = change_search_word_ko_to_en(search_word)
    # MySQL 데이터베이스 연결
    db_connection = connet_db()
    # 커서 객체 생성
    cursor = get_cursor(db_connection)
    # 제품 고유 id 추적
    product_id = find_product_id(cursor, product_name)

    # MySQL에 데이터 삽입
    table_name = f"Specifications_{product_main_name}"  # 테이블 이름 지정
    select_query = f"SELECT COUNT(*) FROM {table_name} WHERE product_id = %s AND spec_name = %s AND spec_value = %s"
    insert_query = f"INSERT INTO {table_name} (product_id, spec_name, spec_value) VALUES (%s, %s, %s)"

    for item in description_content:
        product_id = product_id
        header = item['header']
        value = item['value']
        values = (product_id, header, value)
        # 중복 데이터 확인
        cursor.execute(select_query, values)
        count = cursor.fetchone()[0]
        if count == 0:  # 중복되지 않은 경우에만 삽입
            cursor.execute(insert_query, values)  # 쿼리 실행
            # 변경 사항 커밋
            db_connection.commit()
            print(f"Inserted: {product_id} - {header} - {value}")
        else:
            print(f"Skipped (Duplicate): {product_id} - {header} - {value}")
    print("All data has been inserted into the database.")
    # 커서와 연결 종료
    cursor.close()
    db_connection.close()

def save_python_to_mysql_specification(search_word, start):
    product_main_name = change_search_word_ko_to_en(search_word)
    counts = get_json_data(product_main_name, start) # 100개 데이터 40분
    for count in range(len(counts)):
        try:
            product_name, description_content = get_detail_spec_from_selenium(search_word, count, start)
            insert_selenium_data_to_mysql(search_word, product_name, description_content)
        except:
            print(f"Data has NoneType, So this count number({count}) is pass in the process")

def run_save_python_to_mysql_specifiaction(search_word, start):
    if __name__=="__main__":
        save_python_to_mysql_specification(search_word, start)

In [50]:
def total_run_for_naver_shopping_data(search_word, start):
    run_naver_api(search_word, start)
    run_connect_python_to_mysql_for_products_data(search_word, start)
    run_save_python_to_mysql_specifiaction(search_word, start)
    print("All process are done!")

In [52]:
search_word = "테블릿PC"
start=1
total_run_for_naver_shopping_data(search_word, start)

필터링 후 JSON 파일이 저장되었습니다.
Skipped (Duplicate): 3 - [최종혜택가 60만]삼성전자 갤럭시탭 S9FE 플러스 256GB - 삼성전자 - 갤럭시탭
Skipped (Duplicate): 3 - 삼성전자 갤럭시탭 A9 Galaxy tab A9 - 삼성전자 - 삼성
Inserted: 3 - [최종혜택가 128만+바꿔보상]삼성전자 갤럭시탭 S10 울트라 256GB - 삼성전자 - 갤럭시탭
Skipped (Duplicate): 3 - 삼성전자 갤럭시탭 갤럭시탭S7 FE (SM-T733N) Wi-Fi - 삼성전자 - 갤럭시탭
Inserted: 3 - 삼성 갤럭시탭A9 64G 8.7인치 (SM-X115) LTE - 삼성전자 - 갤럭시탭
Inserted: 3 - 포유디지탈 아이뮤즈 뮤패드 GS10 WIFI 32GB - 포유디지탈 - 아이뮤즈
Inserted: 3 - 애플 아이패드10세대 64GB WIFI 실버색상(MPQ03KH/A) - Apple - Apple
Inserted: 3 - 레노버 리전 패드 Y700 3세대 태블릿 -  - 레노버
Inserted: 3 - Apple 아이패드 10세대 WIFI 64G 블루 - Apple - Apple
Inserted: 3 - 삼성 갤럭시탭A9 X110 8.7인치 64GB 128GB - 삼성전자 - 갤럭시
Inserted: 3 - 삼성전자 갤럭시탭 S9 플러스 512GB - 삼성전자 - 갤럭시탭
Skipped (Duplicate): 3 - Apple 아이패드 프로 13 7세대 M4 - Apple - Apple
Inserted: 3 - 아이뮤즈 12.6인치 안드로이드 태블릿PC, 뮤패드 K13 - 포유디지탈 - 아이뮤즈
Inserted: 3 - 삼성전자 갤럭시탭 S9 울트라 512GB WIFI - 삼성전자 - 갤럭시탭
Inserted: 3 - 갤럭시 탭S7 플러스 12.4 중고 태블릿 - 갤럭시 - 갤럭시
Inserted: 3 - 샤오미 레드미 패드 SE 태블릿 미패드 - 샤오미 - 샤오미
Inserted